



# NLP ASSIGNMENT
Shamiso Makainganwa
R204437W

In [5]:
#importing and loading  all the libraries used in this assignment
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



LOADING THE SHONA TEXT

In [1]:
text = """ Vaitaura vachindinongedza nemudonzvo
wavo.
Vanhu vose vakati bvuu kuseka. Ndaida
kupindura asi ndakanzwa vanhu vakuita
mahon'era ndikaona kuti ndaigona kusara
pachena. Baba vanga vagara pachituro
chavo vakabata shaya. Mhamha
vachipedza kuchema vakabva vashamisa
muromo wavo ndokutanga kutaura.
"Pamusoroyi hama dzangu. Muchiona
muroora wangu maChuma akagara apo,
handina rugare pano pamusha pangu.
Ndakatanga ndichimusiya achiita
zvaanoda nekutaura zvaanoda. Nhasi
chaiye ndamutuma kuchigayo
ndikaudzwa meno amire. Ayenda
naChido mwana wangu vavamunzira
akakamura pagedhi rake Chido akasenga
bhagedhi nehafu iye ndokusenga hafu.
Pavadzoka awuya achizvivharira mumba
make kwanzi ndaneta. Saka apa
ndazomumutsa ndichiti awane chekudya
nekuti anga asina kumbodya masikati
semunhu akazvitakura. Ndazongofira
kubvunza kuti ko asi warwara here kurara
masikati ndipo paatanga kundituka.
Pandangoti nditaurewo abva tanga
kundikwapaidza mambama chaiwo.
Mambama vanhu vaMwari hiiihiihiiii. """

In [2]:
text

' Vaitaura vachindinongedza nemudonzvo\nwavo.\nVanhu vose vakati bvuu kuseka. Ndaida\nkupindura asi ndakanzwa vanhu vakuita\nmahon\'era ndikaona kuti ndaigona kusara\npachena. Baba vanga vagara pachituro\nchavo vakabata shaya. Mhamha\nvachipedza kuchema vakabva vashamisa\nmuromo wavo ndokutanga kutaura.\n"Pamusoroyi hama dzangu. Muchiona\nmuroora wangu maChuma akagara apo,\nhandina rugare pano pamusha pangu.\nNdakatanga ndichimusiya achiita\nzvaanoda nekutaura zvaanoda. Nhasi\nchaiye ndamutuma kuchigayo\nndikaudzwa meno amire. Ayenda\nnaChido mwana wangu vavamunzira\nakakamura pagedhi rake Chido akasenga\nbhagedhi nehafu iye ndokusenga hafu.\nPavadzoka awuya achizvivharira mumba\nmake kwanzi ndaneta. Saka apa\nndazomumutsa ndichiti awane chekudya\nnekuti anga asina kumbodya masikati\nsemunhu akazvitakura. Ndazongofira\nkubvunza kuti ko asi warwara here kurara\nmasikati ndipo paatanga kundituka.\nPandangoti nditaurewo abva tanga\nkundikwapaidza mambama chaiwo.\nMambama vanhu vaMwari hii

PREPROCESSING TEXT

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# Preprocess the text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join the words back into a cleaned text
    cleaned_text = ' '.join(words)

    return cleaned_text

cleaned_text = clean_text(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
cleaned_text

'vaitaura vachindinongedza nemudonzvo wavo vanhu vose vakati bvuu kuseka ndaida kupindura asi ndakanzwa vanhu vakuita mahonera ndikaona kuti ndaigona kusara pachena baba vanga vagara pachituro chavo vakabata shaya mhamha vachipedza kuchema vakabva vashamisa muromo wavo ndokutanga kutaura pamusoroyi hama dzangu muchiona muroora wangu machuma akagara apo handina rugare pano pamusha pangu ndakatanga ndichimusiya achiita zvaanoda nekutaura zvaanoda nhasi chaiye ndamutuma kuchigayo ndikaudzwa meno amire ayenda nachido mwana wangu vavamunzira akakamura pagedhi rake chido akasenga bhagedhi nehafu iye ndokusenga hafu pavadzoka awuya achizvivharira mumba make kwanzi ndaneta saka apa ndazomumutsa ndichiti awane chekudya nekuti anga asina kumbodya masikati semunhu akazvitakura ndazongofira kubvunza kuti ko asi warwara kurara masikati ndipo paatanga kundituka pandangoti nditaurewo abva tanga kundikwapaidza mambama chaiwo mambama vanhu vamwari hiiihiihiiii'

In [8]:
from keras.preprocessing.text import Tokenizer

# Tokenize and preprocess
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
total_words = len(tokenizer.word_index) + 1




In [9]:
from keras.preprocessing.text import text_to_word_sequence
# tokenize the document
result = text_to_word_sequence(text)
print(result)

['vaitaura', 'vachindinongedza', 'nemudonzvo', 'wavo', 'vanhu', 'vose', 'vakati', 'bvuu', 'kuseka', 'ndaida', 'kupindura', 'asi', 'ndakanzwa', 'vanhu', 'vakuita', "mahon'era", 'ndikaona', 'kuti', 'ndaigona', 'kusara', 'pachena', 'baba', 'vanga', 'vagara', 'pachituro', 'chavo', 'vakabata', 'shaya', 'mhamha', 'vachipedza', 'kuchema', 'vakabva', 'vashamisa', 'muromo', 'wavo', 'ndokutanga', 'kutaura', 'pamusoroyi', 'hama', 'dzangu', 'muchiona', 'muroora', 'wangu', 'machuma', 'akagara', 'apo', 'handina', 'rugare', 'pano', 'pamusha', 'pangu', 'ndakatanga', 'ndichimusiya', 'achiita', 'zvaanoda', 'nekutaura', 'zvaanoda', 'nhasi', 'chaiye', 'ndamutuma', 'kuchigayo', 'ndikaudzwa', 'meno', 'amire', 'ayenda', 'nachido', 'mwana', 'wangu', 'vavamunzira', 'akakamura', 'pagedhi', 'rake', 'chido', 'akasenga', 'bhagedhi', 'nehafu', 'iye', 'ndokusenga', 'hafu', 'pavadzoka', 'awuya', 'achizvivharira', 'mumba', 'make', 'kwanzi', 'ndaneta', 'saka', 'apa', 'ndazomumutsa', 'ndichiti', 'awane', 'chekudya', 'ne

In [11]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)




 BUILDING THE VOCABULARY

In [12]:
#using a keras tokenizer to build an optimal vocabulary
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
max_vocab_size= 10000  # the vocabulary size
word_index = tokenizer.word_index
vocabulary_size = min(max_vocab_size, len(word_index))


reduced_word_index = {}
for word, index in word_index.items():
    if index <= vocabulary_size:
        reduced_word_index[word] = index
    else:
        break



tokenizer.word_index = reduced_word_index
tokenizer.word_index[tokenizer.oov_token] = vocabulary_size + 1
tokenizer.num_words = vocabulary_size + 1
vocabulary_size = len(word_index)
print("Vocabulary size:", vocabulary_size)

Vocabulary size: 23


WORD EMBEDDINGS

In [16]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = result
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model.save("shona_embeddings.model")


# BI-DIRECTIONAL MODEL WITH EMBEDDING LAYER

In [17]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 100)            11300     
                                                                 
 bidirectional_1 (Bidirecti  (None, 5, 300)            301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 5, 300)            0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_1 (Dense)             (None, 113)               11413     
                                                                 
Total params: 484313 (1.85 MB)
Trainable params: 484313 (1.85 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [19]:
model1.save('model1.h1')

BI-DIRECTIONAL MODEL WITH PRETRAINED WORD EMBEDDINGS

In [20]:
# Load the word2vec model
import numpy as np
model = Word2Vec.load("shona_embeddings.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = reduced_word_index[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:
        # Word not present in gensim model, a zero embedding will be used for this word
        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 5, 100)            11300     
                                                                 
 bidirectional_2 (Bidirecti  (None, 5, 300)            301200    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 5, 300)            0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 113)               11413     
                                                                 
Total params: 484313 (1.85 MB)
Trainable params: 473013 (1.80 MB)
Non-trainable params: 11300 (44.14 KB)
_______________

In [21]:
model2.save('model2.h1')

In [22]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('model1.h1')
model = load_model('model2.h1')



 MODEL TRAINING

In [23]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in cleaned_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42)

print('training model1')

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=150, verbose=1)

print('training model2')

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=150, verbose=1)

ValueError: ignored

# MODEL TESTING

In [ ]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model1: {val_loss_model1}")
print(f"Validation Loss for Model2: {val_loss_model2}")


Validation Loss for Model1: 11.782182693481445
Validation Loss for Model2: 10.130215644836426


In [ ]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_performing_model1.h5"
else:
    best_model = model2
    best_model_name = "best_performing_model2.h5"

best_model.save(best_model_name)
print(f"the best model is {best_model_name}")


the best model is best_performing_model2.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_performing_model2.h5')

def predict_next_word(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.

    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')

        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)

        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break

        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])




# Prompt the user for input


# Predict the next words


In [ ]:
while(True):

  text = input("Nyora manzwi mashanu: ")

  if text == "0":
      print("chiitiko ichi chaperera pano .....")
      break

  else:
   try:
    user_input = text.split(" ")
    user_input = text[-5:]
    print(user_input)

    # Prompt the user for input
    predicted_word = predict_next_word(model, tokenizer, user_input, num_words=1)
    # Predict the next word
    print(f"inzwi rawanikwa rinoteera manzwi amaisa ndeiro: {predicted_word}")




   except Exception as e:
    print("Error: ", e)
    continue

Nyora manzwi mashanu: manga chena inoparira parere nhema
nhema
inzwi rawanikwa rinoteera manzwi amaisa ndeiro: nhamo
Nyora manzwi mashanu: 0
chiitiko ichi chaperera pano .....
